In [1]:
import pandas as pd
import os
from utils import *
from aggregations import *

In [2]:
data_dict = {
    "pib_par_industrie": {
        "path": os.path.join("data", "stats_can", "pib_par_industrie.csv"),
        "gran:": "industrie and monthly",
        "date_range": ["199701", "202312"],
        "nom": "pib_par_industrie",
        "delimiter": ",",
        "select": {
            "REF_DATE": "CalendarMonth",
            "North American Industry Classification System (NAICS)": "industry",
            "VALUE": "PIB_par_industrie",
        }
    },
    "investissement_construction": {
        "path": os.path.join("data", "stats_can", "investissement_construction.csv"),
        "gran:": "monthly",
        "date_range": ["201701", "202312"],
        "nom": "investissement_construction",
        "delimiter": ";",
        "select": {
            "PÉRIODE DE RÉFÉRENCE": "CalendarMonth",
            "Type de structure": "structure_type",
            "Type de travaux": "work_type",
            "VALEUR": "inverstissement_construction"
        }
    },
    "construction_par_region": {
        "path": os.path.join("data", "stats_can", "construction_par_region.csv"),
        "gran:": "yearly, par region",
        "date_range": ["200501", "202312"],
        "nom": "construction_par_region",
        "delimiter": ";",
        "select": {
            "PÉRIODE DE RÉFÉRENCE": "CalendarYear",
            "Estimations de logement": "construction_status",
            "Type d'unité": "construction_unit_type",
            "VALEUR": "unit_count",
        }
    },
    "indice_de_prix_logements": {
        "path": os.path.join("data", "stats_can", "indice_de_prix_logements.csv"),
        "gran:": "monthly",
        "date_range": ["200501", "202312"],
        "nom": "indice_de_prix_logements",
        "delimiter": ";",
        "select": {
            "PÉRIODE DE RÉFÉRENCE": "CalendarMonth",
            "Indices des prix des logements neufs": "new_housing_price_index",
            "VALEUR": "indice_de_prix_logements",
        }
    },
    "taux_hypothecaire_terme_5ans": {
        "path": os.path.join("data", "stats_can", "taux_hypothecaire_terme_5ans.csv"),
        "gran:": "monthly",
        "date_range": ["200501", "202312"],
        "nom": "taux_hypothecaire_terme_5ans",
        "delimiter": ";",
        "select": {
            "PÉRIODE DE RÉFÉRENCE": "CalendarMonth",
            "VALEUR": "taux_hypothecaire_terme_5ans",
        }
    },
    "demographic_growth":{
        "path": os.path.join("data", "stats_can", "demographic_growth.csv"),
        "gran": "yearly",
        "date_range": ["1998", "2023"],
        "nom": "demographic_growth",
        "delimiter": ",",
        "select": {
            "REF_DATE": "REF_DATE",
            "Components of population growth": "Components of population growth",
            "VALUE": "VALUE",
        }
    }    
}

In [3]:
output_dict = {}
id = 0
for table, table_info in data_dict.items():
    id += 1
    df = load_table(table_info["path"], table_info["delimiter"])
    df = select_rename_columns(df, table_info["select"])
    output_dict[table] = df
    # print the name of the table
    print(" " * 100)    
    print(table)
    if table == "pib_par_industrie":
        print(df.columns)
        df = extract_year(df)        
        df = agg_pib(df)
    if table == "investissement_construction":
        print(df.columns)
        df = extract_year(df)
        df = agg_investissement(df)
    if table == "construction_par_region":
        print(df.columns)
        df = extract_year(df)
        df = agg_construction(df)
    if table == "indice_de_prix_logements":
        print(df.columns)
        df = extract_year(df)
        df = agg_price_index(df)
    if table == "demographic_growth":
        print(df.columns)
        df = extract_year(df)
        df = agg_demo(df)
    # print(get_date_range(df))
    # print(df.shape[0])
    # print(df.head(5))
    print(df.columns)
    df = extract_year(df)

    output_dict[table] = df

                                                                                                    
pib_par_industrie
Index(['CalendarMonth', 'industry', 'PIB_par_industrie'], dtype='object')
Index(['CalendarMonth', 'All industries [T001]', 'Construction [23]',
       'Energy sector [T016]', 'Finance and insurance [52]',
       'Public Sector [T018]', 'Real estate and rental and leasing [53]',
       'Transportation and warehousing [48-49]'],
      dtype='object', name='industry')
                                                                                                    
investissement_construction
Index(['CalendarMonth', 'structure_type', 'work_type',
       'inverstissement_construction'],
      dtype='object')
Index(['CalendarMonth', 'inverstissement_construction'], dtype='object')
                                                                                                    
construction_par_region
Index(['CalendarYear', 'construction_status', 'construction_unit_type

In [4]:

joined_df = output_dict["pib_par_industrie"].merge(output_dict["investissement_construction"], on="CalendarMonth", how="outer")
joined_df["CalendarYear"] = joined_df["CalendarMonth"].apply(lambda x: x[:4])
joined_df["CalendarYear"] = joined_df["CalendarYear"].astype(int)
joined_df = joined_df.merge(output_dict["construction_par_region"], on="CalendarYear", how="outer")
joined_df = joined_df.merge(output_dict["indice_de_prix_logements"], on="CalendarMonth", how="outer")
joined_df = joined_df.merge(output_dict["taux_hypothecaire_terme_5ans"], on="CalendarMonth", how="outer")
joined_df = joined_df.merge(output_dict["demographic_growth"], on="CalendarYear", how="outer")


rename_list = [
    ("All industries [T001]", "PIB_all_industries"),
    ("Construction [23]", "PIB_construction"),
    ("Energy sector [T016]", "PIB_energy"),
    ("Finance and insurance [52]", "PIB_finance_insurance"),
    ("Public Sector [T018]", "PIB_public_sector"),
    ("Real estate and rental and leasing [53]", "PIB_real_estate"),
    ("Transportation and warehousing [48-49]", "PIB_transportation"),
    ("inverstissement_construction", "inverstissement_construction"),
    ("unit_count", "construction_unit_count"),
    ("Maison seulement", "house_only_price_index"),
    ("Terrain seulement", "land_only_price_index"),
    ("Total (maison et terrain)", "price_index_total"),
    ]
for x in rename_list:
    joined_df = joined_df.rename(columns={x[0]: x[1]})

In [5]:
print(joined_df.columns)
print(joined_df.shape[0])

Index(['CalendarMonth', 'PIB_all_industries', 'PIB_construction', 'PIB_energy',
       'PIB_finance_insurance', 'PIB_public_sector', 'PIB_real_estate',
       'PIB_transportation', 'inverstissement_construction', 'CalendarYear',
       'construction_unit_count', 'house_only_price_index',
       'land_only_price_index', 'price_index_total',
       'taux_hypothecaire_terme_5ans', 'Births', 'Deaths', 'Emigrants',
       'Immigrants'],
      dtype='object')
324


In [6]:
joined_df

,CalendarMonth,PIB_all_industries,PIB_construction,PIB_energy,PIB_finance_insurance,PIB_public_sector,PIB_real_estate,PIB_transportation,inverstissement_construction,CalendarYear,construction_unit_count,house_only_price_index,land_only_price_index,price_index_total,taux_hypothecaire_terme_5ans,Births,Deaths,Emigrants,Immigrants
0,1997-01,1204015.0,77365.0,103517.0,67302.0,NaN,137687.0,55400.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997-02,1211631.0,77934.0,104017.0,68039.0,NaN,137446.0,56001.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997-03,1211507.0,77959.0,104637.0,67991.0,NaN,137405.0,56321.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997-04,1219735.0,78973.0,105778.0,67894.0,NaN,137247.0,56666.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997-05,1224088.0,79937.0,104089.0,68419.0,NaN,137310.0,57128.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,2023-08,2201064.0,163953.0,147172.0,157155.0,461762.0,292882.0,91214.0,NaN,2023,NaN,128.1,117.0,125.0,6.17,357903.0,330379.0,94576.0,468817.0
320,2023-09,2200484.0,164288.0,145532.0,156815.0,462571.0,292946.0,91404.0,NaN,2023,NaN,127.8,116.8,124.7,6.27,357903.0,330379.0,94576.0,468817.0
321,2023-10,2201672.0,163725.0,147026.0,157050.0,463230.0,292573.0,91297.0,NaN,2023,NaN,127.6,116.9,124.7,6.42,357903.0,330379.0,94576.0,468817.0
322,2023-11,2205683.0,163047.0,150030.0,156790.0,460454.0,292929.0,92009.0,NaN,2023,NaN,127.2,116.8,124.4,6.47,357903.0,330379.0,94576.0,468817.0


In [7]:
save_table(joined_df, "joined_data")

'joined_data saved at cleaned_data\\joined_data.csv'

In [10]:
test_df = load_table("cleaned_data\joined_data.csv", ",")
test_df

,CalendarMonth,PIB_all_industries,PIB_construction,PIB_energy,PIB_finance_insurance,PIB_public_sector,PIB_real_estate,PIB_transportation,inverstissement_construction,CalendarYear,construction_unit_count,house_only_price_index,land_only_price_index,price_index_total,taux_hypothecaire_terme_5ans,Births,Deaths,Emigrants,Immigrants
0,1997-01,1204015.0,77365.0,103517.0,67302.0,NaN,137687.0,55400.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997-02,1211631.0,77934.0,104017.0,68039.0,NaN,137446.0,56001.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997-03,1211507.0,77959.0,104637.0,67991.0,NaN,137405.0,56321.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997-04,1219735.0,78973.0,105778.0,67894.0,NaN,137247.0,56666.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997-05,1224088.0,79937.0,104089.0,68419.0,NaN,137310.0,57128.0,NaN,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,2023-08,2201064.0,163953.0,147172.0,157155.0,461762.0,292882.0,91214.0,NaN,2023,NaN,128.1,117.0,125.0,6.17,357903.0,330379.0,94576.0,468817.0
320,2023-09,2200484.0,164288.0,145532.0,156815.0,462571.0,292946.0,91404.0,NaN,2023,NaN,127.8,116.8,124.7,6.27,357903.0,330379.0,94576.0,468817.0
321,2023-10,2201672.0,163725.0,147026.0,157050.0,463230.0,292573.0,91297.0,NaN,2023,NaN,127.6,116.9,124.7,6.42,357903.0,330379.0,94576.0,468817.0
322,2023-11,2205683.0,163047.0,150030.0,156790.0,460454.0,292929.0,92009.0,NaN,2023,NaN,127.2,116.8,124.4,6.47,357903.0,330379.0,94576.0,468817.0
